# L'évolution de la population scolaire et de la population totale à Paris entre 2019 et 2021, par arrondissement 
### Problématique : Est-ce que l'évolution de la démographie des écoles est en accord avec évolution de la population (par classe d'âge) ?

Claire du Campe de Rosamel, Benoît Nallet, Amandine Perret

## 1. Introduction



Avant de commencer l'analyse on aura besoin des packages suivants : 

In [2]:
!pip install unidecode
!pip install py7zr geopandas openpyxl tqdm s3fs 
!pip install PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette
!pip install urllib3==1.26.5
!pip install prettytable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 5.1 MB/s eta 0:00:00a 0:00:01
  Cloning https://github.com/inseefrlab/cartiflette to /tmp/pip-req-build-mks2ev1e
  Running command git clone --filter=blob:none --quiet https://github.com/inseefrlab/cartiflette /tmp/pip-req-build-mks2ev1e
  Resolved https://github.com/inseefrlab/cartiflette to commit f6b0c5d049bd6dcae7b437bd931957371da24278
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 9.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 26.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 15.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for cartiflette: filename=cartiflette-0.0.2-py3-none-any.whl size=56573 sha256=7e59f26a71f86d2e94fde43e6

In [ ]:
import pandas as pd
import zipfile
import requests
import os
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from unidecode import unidecode 
import matplotlib.ticker as mticker
import geopandas as gpd
from cartiflette import carti_download

Quelques définitions :
Les unités statistiques de notre étude sont :

- la population en logement ordinaire et en résidence principale,
- la population scolaire en école primaire (maternelle et élémentaire).

Il est possible qu'un élève soit scolarisé dans un autre arrondissement que celui où il réside. Il faudra donc distinguer la populaire scolaire dénombrée au niveau de son lieu de résidence principale de celle dénombrée au niveau de son lieu de scolarisation.

Le champ de l'étude est restreint à la ville de Paris pour les années 2019 à 2021.

## 2. Collecte et nettoyage des données 

Notre analyse se fait à partir des plusieurs bases de données publique issues de la Statistique publique et disponible directement sur le site internet de l'Insee.
 
Les effectifs scolaires sont disponibles à l'url XX recensant pour toute la France le nombre d'élèves par classe et par école. Nous avons donc réduit l'étendue de la base de données aux seules écoles dans Paris intra-muros (dont le code postal commence par 75). 

Une deuxième partie des données est issue du recensement fait par l'Insee. Plusieurs bases de données ont été nécessaires : la base logement pour comptabiliser la population habitant effectivement dans chaque arrondissement.

Enfin la base XY renseigne les effectifs de population par tranche d'âge.

Les différentes bases de données sont importées directement depuis leur adresse internet et sont ensuite nettoyées dans le script "import_data.py" ci-dessous.

In [ ]:
from scripts import import_data

## 3. Quels constats /!\ A la fin il faudra enlever les print du script

Depuis la crise sanitaire la ville de Paris a perdu des habitants dans un contexte de vieillissement de la population.

Ces pertes sont concentrées dans le Nord-Est parisien et on retrouve la même géographie de perte pour la population par arrondissement de scolarisation.

Pourtant les fermetures de classes/d'écoles se sont concentrées dans XXXZZZ.


In [ ]:
from scripts import stats_descr_recensement

In [ ]:
from scripts import stats_descr_effectif ;

### A. L'évolution de la population à Paris par arrondissement entre 2019 et 2021
#### a. L'évolution globale 

In [ ]:
print("Tableau de l'évolution globale de la population en résidence principale parisienne")
print(stats_descr_recensement.pop_totale)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.figure(figsize=(10, 6))
plt.imshow(mpimg.imread("graphs/carte_evol_population_niveau.png"))
plt.axis("off")
print("attention légende fausse")

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(mpimg.imread("graphs/carte_evol_population_pourcentage.png"))
plt.axis("off")
print("attention légende fausse")


#### b. Focus sur la pyramide des âges 



In [ ]:
from scripts import stats_ages

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(mpimg.imread("graphs/Pyramide_Paris_ecart1921.png"))
plt.title("Evolution de la pyramide des âges entre 2019 et 2021")
plt.axis("off")
print("attention légende fausse")

### B. L'évolution des effectifs scolaires par arrondissement entre 2019 et 2021
#### a. Au global 


In [ ]:
print("Tableau de la baisse du nombre d'élèves")
print(stats_descr_effectif.table_pertes_df)

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(mpimg.imread("graphs/carte_evol_effectifs_niveau.png"))
plt.axis("off")

In [ ]:
plt.figure(figsize=(10, 6))
plt.imshow(mpimg.imread("graphs/carte_evol_effectifs_pourcentage.png"))
plt.axis("off")
print("ca serait utile de mettre le chiffre associé à la borne de droite de la légende")


#### b. La population en primaire

#### c. La population en maternelle 

`Pour l'instant mes cellules tournent, les scripts fonctionnent et on peut normalement récupérer les graphiques et des bases de données comme dans les exemples plus haut`

%matplotlib inline permet que quand le script est importé, tous les graphiques ne s'affichent pas

## 4. Modelisation 

Ici les regressions d'Amandine 

In [1]:
from scripts import regressions

       ARM  is_T3  is_bac+5
0  75101.0  False     False
1  75101.0   True     False
2  75101.0  False     False
3  75101.0  False      True
4  75101.0  False     False
        ARM  Proportion_3_pieces_ou_plus  Proportion_bac+5
0   75101.0                     0.292504          0.278525
1   75102.0                     0.256221          0.340537
2   75103.0                     0.290036          0.355793
3   75104.0                     0.303363          0.314031
4   75105.0                     0.344104          0.393335
5   75106.0                     0.345176          0.348443
6   75107.0                     0.342441          0.349192
7   75108.0                     0.344595          0.310901
8   75109.0                     0.336761          0.375933
9   75110.0                     0.314613          0.322677
10  75111.0                     0.305554          0.337951
11  75112.0                     0.363464          0.298753
12  75113.0                     0.398926          0.292613
13  75

/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/scripts/regressions.py:121: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ages_2021 = pd.read_csv('/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/extracted_files/TD_POP1B_2021.csv', delimiter=';', encoding='latin1')
/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/scripts/regressions.py:122: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ages_2020 = pd.read_csv('/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/extracted_files/TD_POP1B_2020.csv', delimiter=';', encoding='latin1')
/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/scripts/regressions.py:123: DtypeW

    CODGEO  Proportion_30_40_Ans
0    75101              0.165788
1    75102              0.201699
2    75103              0.186804
3    75104              0.158299
4    75105              0.124023
5    75106              0.119869
6    75107              0.121737
7    75108              0.136073
8    75109              0.195379
9    75110              0.202234
10   75111              0.199982
11   75112              0.169881
12   75113              0.154218
13   75114              0.149984
14   75115              0.163298
15   75116              0.120079
16   75117              0.176450
17   75118              0.206879
18   75119              0.165693
19   75120              0.173475
    CODGEO  Proportion_30_40_Ans
0    75101              0.164980
1    75102              0.195695
2    75103              0.181127
3    75104              0.157591
4    75105              0.122420
5    75106              0.120267
6    75107              0.119641
7    75108              0.134635
8    75109

In [3]:
import pandas as pd
print(pd.DataFrame({ "Coefficients": regressions.model_2019.fit().params, "P-value": regressions.model_2019.fit().pvalues}))
print(pd.DataFrame({ "Coefficients": regressions.model_2020.fit().params, "P-value": regressions.model_2020.fit().pvalues}))
print(pd.DataFrame({ "Coefficients": regressions.model_2021.fit().params, "P-value": regressions.model_2021.fit().pvalues}))

                              Coefficients   P-value
const                        -37741.273346  0.008736
Proportion_3_pieces_ou_plus  101262.973127  0.000013
Proportion_bac+5             -13358.900523  0.471833
Proportion_30_40_Ans          89961.804451  0.003992
                              Coefficients   P-value
const                        -35998.655415  0.014571
Proportion_3_pieces_ou_plus  100188.857085  0.000019
Proportion_bac+5             -14040.881086  0.478561
Proportion_30_40_Ans          83854.491448  0.006106
                             Coefficients   P-value
const                       -36625.780468  0.009620
Proportion_3_pieces_ou_plus  98730.135869  0.000013
Proportion_bac+5             -9144.962215  0.622941
Proportion_30_40_Ans         80279.477790  0.005406


In [7]:
from scripts import regressions_effectifs

       ARM  is_T3  is_bac+5
0  75101.0  False     False
1  75101.0   True     False
2  75101.0  False     False
3  75101.0  False      True
4  75101.0  False     False
        ARM  Proportion_3_pieces_ou_plus  Proportion_bac+5
0   75101.0                     0.292504          0.278525
1   75102.0                     0.256221          0.340537
2   75103.0                     0.290036          0.355793
3   75104.0                     0.303363          0.314031
4   75105.0                     0.344104          0.393335
5   75106.0                     0.345176          0.348443
6   75107.0                     0.342441          0.349192
7   75108.0                     0.344595          0.310901
8   75109.0                     0.336761          0.375933
9   75110.0                     0.314613          0.322677
10  75111.0                     0.305554          0.337951
11  75112.0                     0.363464          0.298753
12  75113.0                     0.398926          0.292613
13  75

/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/scripts/regressions_effectifs.py:131: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ages_2021 = pd.read_csv('extracted_files/TD_POP1B_2021.csv', delimiter=';', encoding='latin1')
/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/scripts/regressions_effectifs.py:132: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ages_2020 = pd.read_csv('extracted_files/TD_POP1B_2020.csv', delimiter=';', encoding='latin1')
/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/scripts/regressions_effectifs.py:133: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ages_2019 = pd.read_csv('extracted_files/BTT_TD_POP1B_2019.csv', delimiter=

    CODGEO  Proportion_30_40_Ans
0    75101              0.165788
1    75102              0.201699
2    75103              0.186804
3    75104              0.158299
4    75105              0.124023
5    75106              0.119869
6    75107              0.121737
7    75108              0.136073
8    75109              0.195379
9    75110              0.202234
10   75111              0.199982
11   75112              0.169881
12   75113              0.154218
13   75114              0.149984
14   75115              0.163298
15   75116              0.120079
16   75117              0.176450
17   75118              0.206879
18   75119              0.165693
19   75120              0.173475
    CODGEO  Proportion_30_40_Ans
0    75101              0.164980
1    75102              0.195695
2    75103              0.181127
3    75104              0.157591
4    75105              0.122420
5    75106              0.120267
6    75107              0.119641
7    75108              0.134635
8    75109

In [8]:
import pandas as pd
print(pd.DataFrame({ "Coefficients": regressions_effectifs.model_effectifs_absolus.fit().params, "P-value": regressions_effectifs.model_effectifs_absolus.fit().pvalues}))

                             Coefficients       P-value
const                        14623.218536  2.183783e-03
Proportion_3_pieces_ou_plus   8902.797092  2.817066e-01
Proportion_bac+5             -8264.344366  3.142424e-01
Proportion_30_40_Ans        -13003.245768  4.534329e-01
arrondissement_75101        -11831.083512  3.220150e-14
arrondissement_75102        -10365.289765  1.497030e-07
arrondissement_75103        -10152.826598  1.296519e-08
arrondissement_75104        -10558.678113  1.616610e-12
arrondissement_75105         -9214.242981  2.349431e-07
arrondissement_75106        -10244.420425  3.511745e-10
arrondissement_75107         -9674.190765  2.096412e-09
arrondissement_75108         -9920.318787  3.739222e-13
arrondissement_75109         -8350.161580  1.040644e-07
arrondissement_75110         -6111.206278  3.825345e-06
arrondissement_75111         -3166.697326  1.673204e-02
arrondissement_75112         -3752.912536  1.329150e-09
arrondissement_75113         -1173.085684  8.781

In [4]:
from scripts import regressions_ratios

       ARM  is_T3  is_bac+5
0  75101.0  False     False
1  75101.0   True     False
2  75101.0  False     False
3  75101.0  False      True
4  75101.0  False     False
        ARM  Proportion_3_pieces_ou_plus  Proportion_bac+5
0   75101.0                     0.292504          0.278525
1   75102.0                     0.256221          0.340537
2   75103.0                     0.290036          0.355793
3   75104.0                     0.303363          0.314031
4   75105.0                     0.344104          0.393335
5   75106.0                     0.345176          0.348443
6   75107.0                     0.342441          0.349192
7   75108.0                     0.344595          0.310901
8   75109.0                     0.336761          0.375933
9   75110.0                     0.314613          0.322677
10  75111.0                     0.305554          0.337951
11  75112.0                     0.363464          0.298753
12  75113.0                     0.398926          0.292613
13  75

/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/scripts/regressions_ratios.py:103: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  
/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/scripts/regressions_ratios.py:104: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ages_2021 = pd.read_csv('extracted_files/TD_POP1B_2021.csv', delimiter=';', encoding='latin1')
/home/onyxia/work/Projet-Python-evolution-des-ecoles-par-arrondissement-dans-Paris-entre-2019-et-2022/scripts/regressions_ratios.py:105: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ages_2020 = pd.read_csv('extracted_files/TD_POP1B_2020.csv', delimiter=';', encoding='latin1')


    CODGEO  Proportion_30_40_Ans
0    75101              0.165788
1    75102              0.201699
2    75103              0.186804
3    75104              0.158299
4    75105              0.124023
5    75106              0.119869
6    75107              0.121737
7    75108              0.136073
8    75109              0.195379
9    75110              0.202234
10   75111              0.199982
11   75112              0.169881
12   75113              0.154218
13   75114              0.149984
14   75115              0.163298
15   75116              0.120079
16   75117              0.176450
17   75118              0.206879
18   75119              0.165693
19   75120              0.173475
    CODGEO  Proportion_30_40_Ans
0    75101              0.171042
1    75102              0.198457
2    75103              0.183113
3    75104              0.155268
4    75105              0.122031
5    75106              0.122039
6    75107              0.117915
7    75108              0.132271
8    75109

/opt/conda/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/opt/conda/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/opt/conda/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


In [5]:
import pandas as pd
print(pd.DataFrame({ "Coefficients": regressions_ratios.model_avec_ratios.fit().params, "P-value": regressions_ratios.model_avec_ratios.fit().pvalues}))

                      Coefficients  P-value
const                     0.228506      NaN
ratio_3_pieces            0.226026      NaN
ratio_bac+5               0.232660      NaN
ratio_30_40               0.225272      NaN
arrondissement_75101      0.006089      NaN
arrondissement_75102     -0.033272      NaN
arrondissement_75103     -0.021932      NaN
arrondissement_75104     -0.003416      NaN
arrondissement_75105      0.029065      NaN
arrondissement_75106      0.055536      NaN
arrondissement_75107      0.018662      NaN
arrondissement_75108      0.026177      NaN
arrondissement_75109      0.002332      NaN
arrondissement_75110     -0.039326      NaN
arrondissement_75111     -0.004167      NaN
arrondissement_75112      0.017460      NaN
arrondissement_75113      0.045554      NaN
arrondissement_75114      0.028964      NaN
arrondissement_75115      0.039988      NaN
arrondissement_75116      0.042869      NaN
arrondissement_75117      0.043037      NaN
arrondissement_75118     -0.0103

## 5. Conclusion
